# Paper Notes

This notebook summarizes some important question to embedding an image into the laten space of StyleGAN

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/internship/style-gan/

/content/gdrive/MyDrive/internship/style-gan


In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3
!pip install lpips
!pip install pytorch-ignite
!pip install pytorch-msssim

     |████████████████████████████████| 204kB 4.2MB/s 
     |████████████████████████████████| 112kB 24.1MB/s 
     |████████████████████████████████| 26.9MB 109kB/s 
     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 204kB 4.3MB/s 


In [ ]:
# Import the needed libraries
import matplotlib.pyplot as plt
import pickle
import torch
import torch.nn as nn
import os
import IPython
from PIL import Image
import glob
from sklearn.decomposition import PCA
import numpy as np 
from torchvision.utils import save_image
from torchsummary import summary
from torchvision import models, transforms
from ignite.metrics import PSNR
from ignite.engine import Engine
from pytorch_msssim import ssim

import torch.optim as optim
import torch.nn.functional as F
import lpips
import warnings
import dnnlib
import pandas as pd
import PIL


In [ ]:
# Setting global attributes
RESOLUTION = 1024
#DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ITERATIONS = 1300
SAVE_STEP = 100

# OPTIMIZER
LEARNING_RATE = 0.01
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8
regularizer_lambda = 0.001

# IMAGE TO EMBED
#PATH_IMAGE = "stuff/data/expression02.png"
PATH_DIR = "stuff/data/input/"
SAVING_DIR = 'stuff/results/paper_notes/'

In [ ]:
DEVICE

device(type='cuda', index=0)

## Loadding Pretrained Model
Load the pretrained model using the pickle file. I need the libraries `dnnlib` and `torch_utils` to load this model.

It does not need source code for the networks themselves — their class definitions are loaded from the pickle via `torch_utils.persistence`.


In [ ]:
PRETRAINED_MODEL = "stuff/pretrained_models/ffhq.pkl"

with open(PRETRAINED_MODEL, 'rb') as f:
    G = pickle.load(f)['G_ema'].to(DEVICE)  # torch.nn.Module-


The pickle contains three networks. `'G'` and `'D'` are instantaneous snapshots taken during training, and `'G_ema'` represents a moving average of the generator weights over several training steps. The networks are regular instances of `torch.nn.Module`, with all of their parameters and buffers placed on the CPU at import and gradient computation disabled by default.

The generator consists of two submodules, `G.mapping` and `G.synthesis`, that can be executed separately. They also support various additional options:

```.python
w = G.mapping(z, c, truncation_psi=0.5, truncation_cutoff=8)
img = G.synthesis(w, noise_mode='const', force_fp32=True)
```
where

```.python
z: latent_code
c: class label
w: intermediate latent_code
```

Please refer to [`generate.py`](./generate.py), [`style_mixing.py`](https://github.com/NVlabs/stylegan2-ada-pytorch/blob/main/style_mixing.py), and [`projector.py`](./projector.py) for further examples.

From G we need to extract the `mapping` and the `synthesis` modules.

## Sampling


1. Sample (generate) 50 face images (in z-space) using StyleGAN and look at them carefully. What artifacts can you observe? How could you tell an StyleGAN image apart from a real image?

In [ ]:
z_latents = torch.randn([50, G.z_dim]).to(DEVICE)

In [ ]:
truncation_value = 1
output_dir = os.path.join(SAVING_DIR,f'/trunc_{truncation_value}')
if not os.path.exits(output_dir):
  os.makedir(output_dir)

for latent_idx, z in enumerate(z_latents):
    print('Generating image for latent (%d/%d) ...' % (latent_idx, len(z_latents)))
    img = G(z.unsqueeze(0), None, truncation_psi=truncation_value, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    filename = os.path.join(output_dir, f'image_{latent_idx:03d}.png')
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(filename)

2. Sample 50 images without truncation (truncation factor t = 1), 50 images with truncation factor t = 0.7, and to images with truncation factor t = 0.5. What do you observe? Can you verbally describe the differences between the different truncation factors (the truncation factor has another name in the paper, not t).

In [ ]:
truncation_value = 0.7
output_dir = os.path.join(SAVING_DIR,f'/trunc_{truncation_value}')
if not os.path.exits(output_dir):
  os.makedir(output_dir)

for latent_idx, z in enumerate(z_latents):
    print('Generating image for latent (%d/%d) ...' % (latent_idx, len(z_latents)))
    img = G(z.unsqueeze(0), None, truncation_psi=truncation_value, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    filename = os.path.join(output_dir, f'image_{latent_idx:03d}.png')
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(filename)

In [ ]:
truncation_value = 0.5
output_dir = os.path.join(SAVING_DIR,f'/trunc_{truncation_value}')
if not os.path.exits(output_dir):
  os.makedir(output_dir)

for latent_idx, z in enumerate(z_latents):
    print('Generating image for latent (%d/%d) ...' % (latent_idx, len(z_latents)))
    img = G(z.unsqueeze(0), None, truncation_psi=truncation_value, noise_mode='const')
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    filename = os.path.join(output_dir, f'image_{latent_idx:03d}.png')
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(filename)

3. What is the difference between w / w+ or z / z+ space? In w there is only one latent code with 512 floating point variables and in w+ there are 18 latent codes with 512 floating point variables. There are also different spaces like w2, w3, w6, w9. In these other spaces the latent codes are split into groups that can be different. For example, in w2 there are two groups of latent codes (one for the first 9 layers and one for the later 9 layers). Sample k images from z, z2, z3, z6, z9, z18 = z+. What do you observe? Which images look realistic and which do not look realistic?